In [ ]:
structure = "Primary motor area, Layer 2/3"
from pathlib import Path
import os

from allensdk.core.reference_space_cache import ReferenceSpaceCache

output_dir = '.'
reference_space_key = os.path.join('annotation', 'ccf_2017')
rspc = ReferenceSpaceCache(resolution=25, reference_space_key=reference_space_key, manifest=Path(output_dir) / 'manifest.json')
# ID 1 is the adult mouse structure graph
tree = rspc.get_structure_tree(structure_graph_id=1) 
id_acronym_map = tree.get_id_acronym_map()
#tree.get_structures_by_acronym(['MO5']) # if:299
tree.parents([id_acronym_map['MO5']])
from collections import Counter
id_acronym_map = tree.get_id_acronym_map()
ids = [id_acronym_map[acronym] for acronym in list(points_csv['acronym'])]
#tree.get_structures_by_acronym(['MO5']) # if:299
parent_dics = tree.parents(ids)
parent_acronyms = [parent_dict['acronym'] for parent_dict in parent_dics]
parent_counts = dict(Counter(parent_acronyms))
parent_counts

{'LAT': 2,
 'VP': 1,
 'LGd': 2,
 'SSp-bfd': 1,
 'VISrl': 4,
 'VISp': 4,
 'VISal': 1,
 'VISpm': 1,
 'AUDpo': 1,
 'VISl': 1,
 'ENTm': 1}

In [15]:
import pandas as pd
import os
import vedo
vedo.settings.default_backend= 'vtk'
import matplotlib.cm as cm
import numpy as np
from brainrender.actors import Points, PointsDensity
from vedo import Plotter
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from brainrender import Scene,settings
from getCellCount import *
from getCoordData import *
from brainRender import *

count_df = get_count_df(
    '/run/user/1000/gvfs/smb-share:server=data.einsteinmed.edu,share=users/Gianna Mattessich/2P_desktop_data/SliceData/nelson_brains/coord_files/7556_points.csv',
    hemisphere=True)
#acronym_counts, name_counts = get_count_dicts(count_df,hemisphere=True)
counts_data = get_filtered_counts(count_df, area_contains='VIS',hemisphere=True)
print(counts_data)
hemisphere=True
cmap_name = 'Reds'
regions_alpha = 0.8
title= 'title'
markers=False
xml_data=None
marker_radius=None
markers_alpha=None
marker_colors=None
settings.SHADER_STYLE = "metallic" 
settings.SHOW_AXES = False
settings.BACKGROUND_COLOR = 'black'
settings.ROOT_ALPHA = 1
if isinstance(counts_data, pd.DataFrame):
    acronym_counts, name_counts = get_count_dicts(count_df,hemisphere=hemisphere)
elif isinstance(counts_data, dict):
    acronym_counts = counts_data
else:
    raise TypeError('Provided input data is not an acronym count dictionary or a count ')
acronym_color_map = areas_colormap_dict(acronym_counts,cmap_name=cmap_name)
print(acronym_color_map)
scene = Scene(atlas_name='allen_mouse_25um',title=title)
for acronym, hex_color in acronym_color_map.items():
    if hemisphere:
        right_acronym_colors = {acronym:color for acronym,color in acronym_color_map.items() if 'right' in acronym}
        print(right_acronym_colors)
        left_acronym_colors = {acronym:color for acronym,color in acronym_color_map.items() if 'left' in acronym}
        for acronym,hex_color in right_acronym_colors.items():
            areaname, hemisphere = tuple(acronym.split(','))
            scene.add_brain_region(areaname,alpha=regions_alpha,hemisphere='right',color=hex_color,silhouette=True,force=True)
        for acronym,hex_color in left_acronym_colors.items():
            areaname, hemisphere = tuple(acronym.split(','))
            scene.add_brain_region(areaname,alpha=regions_alpha,hemisphere='left',color=hex_color,silhouette=True,force=True)
    for acronym,hex_color in acronym_color_map.items():    
        areaname, hemisphere = tuple(acronym.split(','))
        scene.add_brain_region(areaname,alpha=regions_alpha,hemisphere='both',color=hex_color,force=True)
if markers:
    if xml_data is None:
        raise ValueError('Markers argument was set to True but no XML data was provided. Please provide either a folder containing' \
        'properly constructed XML file, or folder containing XML files')
    positions, markers = get_coords(xml_data)
    marker_19_coords = markers[markers['Marker'] == 'marker 19'].loc[:,['x','y','z']].to_numpy()
    marker_20_coords = markers[markers['Marker'] == 'marker 20'].loc[:,['x','y','z']].to_numpy()
    scene.add(Points(marker_19_coords, radius=marker_radius, alpha=markers_alpha, colors=marker_colors[0]))
    scene.add(Points(marker_20_coords, radius=marker_radius, alpha=markers_alpha, colors=marker_colors[1]))
# render
anim = Animation(scene, save_fld='/home/gianna/Downloads', name='brainrender_animation',fmt='.mp4',size="1280x920")

# Specify camera position and zoom at some key frames
# each key frame defines the scene's state after n seconds have passed
anim.add_keyframe(0, camera="top", zoom=1)
anim.add_keyframe(1.5, camera="sagittal", zoom=0.95)
anim.add_keyframe(3, camera="sagittal", zoom=0.8)
anim.add_keyframe(4, camera="sagittal", zoom=0.9)
anim.add_keyframe(5, camera="frontal", zoom=1)
anim.add_keyframe(6, camera="frontal", zoom=1.2)
anim.add_keyframe(7, camera="top", zoom=1)

# Make videos
anim.make_video(duration=7, fps=15, resetcam=True)


{'VISrl4,right': 8, 'VISp5,right': 20, 'VISp6a,right': 13, 'VISal5,right': 1, 'VISpm5,right': 3, 'VISp2/3,right': 31, 'VISp4,right': 14, 'VISrl6a,right': 8, 'VISrl5,right': 3, 'VISrl2/3,right': 6, 'VISl5,right': 1}
{'VISrl4,right': '#fcc0a8', 'VISp5,right': '#ec382a', 'VISp6a,right': '#fb8a6a', 'VISal5,right': '#fff5f0', 'VISpm5,right': '#fee9e0', 'VISp2/3,right': '#67000c', 'VISp4,right': '#fb7f5f', 'VISrl6a,right': '#fcc0a8', 'VISrl5,right': '#fee9e0', 'VISrl2/3,right': '#fdd4c2', 'VISl5,right': '#fff5f0'}
{'VISrl4,right': '#fcc0a8', 'VISp5,right': '#ec382a', 'VISp6a,right': '#fb8a6a', 'VISal5,right': '#fff5f0', 'VISpm5,right': '#fee9e0', 'VISp2/3,right': '#67000c', 'VISp4,right': '#fb7f5f', 'VISrl6a,right': '#fcc0a8', 'VISrl5,right': '#fee9e0', 'VISrl2/3,right': '#fdd4c2', 'VISl5,right': '#fff5f0'}
{'VISrl4,right': '#fcc0a8', 'VISp5,right': '#ec382a', 'VISp6a,right': '#fb8a6a', 'VISal5,right': '#fff5f0', 'VISpm5,right': '#fee9e0', 'VISp2/3,right': '#67000c', 'VISp4,right': '#fb7f5f'

Saving video in /home/gianna/Downloads

📽  Video file brainrender_animation is open... 

Output()

Saving video

ffmpeg returned an error while trying to save video with command:
    ffmpeg -hide_banner -loglevel panic -y -r 15 -start_number 0 -i /tmp/tmpjaerx2mh/%d.png -vcodec libx264 -crf 28
-pix_fmt yuv420p -s 1280x920 brainrender_animation..mp4

'/home/gianna/Downloads/brainrender_animation..mp4'